In [89]:
import numpy as np

import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone
import sklearn

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam

from aif360.datasets import AdultDataset, BankDataset, CompasDataset, GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms import preprocessing, inprocessing, postprocessing
import art
import fairlearn

from IPython.display import Markdown, display

In [28]:
def run_classification_metrics(CM:ClassificationMetric):
    return np.array([
        round(CM.accuracy(), 4),
        round(CM.theil_index(), 4),
        round(CM.consistency()[0], 4),
        round(CM.false_positive_rate_difference(), 4),
        round(CM.false_negative_rate_difference(), 4),
        round(CM.error_rate_difference(), 4),
        round(CM.false_discovery_rate_difference(), 4),
        round(CM.false_omission_rate_difference(), 4)
    ])

In [29]:
def run_binary_dataset_metrics(BLDM:BinaryLabelDatasetMetric):
    return np.array([
        round(BLDM.statistical_parity_difference(), 4), # negative means privileged bias
        round(BLDM.base_rate(privileged=True), 4), # 1 means privileged bias
        round(BLDM.base_rate(privileged=False), 4), # 1 means unprivileged bias
    ])

In [33]:
def analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, classifier, 
                 preprocessing_algo=None):
    if preprocessing_algo is not None:
        _dataset_train = preprocessing_algo.fit_transform(dataset_train)
    else:
        _dataset_train = dataset_train
    
    classifier.fit(_dataset_train.features, dataset_train.labels.ravel())
    results = classifier.predict(dataset_test.features)
    
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(_dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    return np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

In [102]:
def run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups):
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

    RF = RandomForestClassifier(n_estimators=1100)

    RW = preprocessing.Reweighing(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups)

    DIR = preprocessing.DisparateImpactRemover()

    #OP = preprocessing.OptimPreproc(SGD, {"learning_rate":0.1}, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    LFR = preprocessing.LFR(unprivileged_groups, privileged_groups)

    metrics = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, RF)
    df = pd.DataFrame(metrics, columns=["No Intervention"])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference", "error rate difference",
                "false discovery rate difference", "false omission rate difference",
                "stat parity difference", "priv base rate", "unpriv base rate"]
    df["Reweighing"] = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, RF, RW)
    df["Disparate Impact Remover"] = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, RF, DIR)
    #df["Optimized Preprocessing"] = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, RF, OP)
    df["Learning Fair Representations"] = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, RF, LFR)
    return df

## Preprocessing Algos Analysis

In [103]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("German Dataset"))

dataset = AdultDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Adult Dataset"))

dataset = BankDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['day_of_week'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Bank Dataset"))

dataset = CompasDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]
df = run_preproc_algos_on_dataset(dataset, unprivileged_groups, privileged_groups)
display(df.style.set_caption("Compas Dataset"))

,No Intervention,Reweighing,Disparate Impact Remover,Learning Fair Representations
accuracy,0.743300,0.733300,0.743300,0.696700
theil index,0.111800,0.122700,0.121400,0.067400
consistency,0.664000,0.664000,0.664000,0.664000
false positive rate difference,-0.052800,-0.052800,-0.075000,-0.008300
false negative rate difference,0.043500,0.074300,0.027200,-0.016300
error rate difference,0.098700,0.113600,0.072100,0.150600
false discovery rate difference,0.146000,0.155400,0.130500,0.165900
false omission rate difference,-0.102300,-0.071400,-0.150000,-0.500000
stat parity difference,-0.109800,0.000000,-0.109800,0.000000
priv base rate,0.719300,0.702900,0.719300,1.000000


,No Intervention,Reweighing,Disparate Impact Remover,Learning Fair Representations
accuracy,0.846000,0.845800,0.840900,0.244900
theil index,0.117600,0.117800,0.117100,0.033900
consistency,0.848400,0.848400,0.848400,0.848400
false positive rate difference,-0.090600,-0.090300,-0.107700,0.000000
false negative rate difference,0.059300,0.063300,0.103800,0.000000
error rate difference,-0.121000,-0.120300,-0.124600,0.197400
false discovery rate difference,-0.011600,-0.008400,-0.027200,0.197400
false omission rate difference,-0.103100,-0.102600,-0.096100,0.000000
stat parity difference,-0.199500,-0.000000,-0.199500,-0.014600
priv base rate,0.313500,0.249100,0.313500,0.026800


,No Intervention,Reweighing,Disparate Impact Remover,Learning Fair Representations
accuracy,0.903100,0.902800,0.837400,0.877800
theil index,0.076500,0.076800,0.082000,0.130400
consistency,0.899800,0.899800,0.899800,0.899800
false positive rate difference,0.033900,0.039000,-0.041200,0.000000
false negative rate difference,-0.036700,-0.020700,0.062300,0.000000
error rate difference,0.063200,0.070800,0.003600,0.096700
false discovery rate difference,-0.038000,-0.015900,-0.238000,0.000000
false omission rate difference,0.055400,0.059800,0.061100,0.096700
stat parity difference,0.111100,0.000000,0.111100,0.000000
priv base rate,0.125300,0.128400,0.125300,1.000000


,No Intervention,Reweighing,Disparate Impact Remover,Learning Fair Representations
accuracy,0.644000,0.644500,0.643400,0.454300
theil index,0.244700,0.243400,0.261000,0.788900
consistency,0.661600,0.661600,0.661600,0.661600
false positive rate difference,-0.200700,-0.209300,-0.216600,0.000000
false negative rate difference,0.180900,0.161700,0.210200,0.000000
error rate difference,0.055000,0.040200,0.059200,-0.151600
false discovery rate difference,0.094500,0.086300,0.090300,0.000000
false omission rate difference,-0.080400,-0.106500,-0.067700,-0.151600
stat parity difference,-0.117500,-0.000000,-0.117500,0.000000
priv base rate,0.639100,0.544000,0.639100,1.000000


In [101]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

RF = RandomForestClassifier(n_estimators=1100)
OP = preprocessing.OptimPreproc(SGD, None, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

metrics = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, RF)
df = pd.DataFrame(metrics, columns=["No Intervention"])
df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
            "false negative rate difference", "error rate difference",
            "false discovery rate difference", "false omission rate difference",
            "stat parity difference", "priv base rate", "unpriv base rate"]
df["Optimized Preprocessing"] = analyze_algo(dataset, dataset_test, privileged_groups, unprivileged_groups, RF, OP)


Privileged and unprivileged groups specified will not be used. The protected attributes are directly specified in the data preprocessing function. The current implementation automatically adjusts for discrimination across all groups. This can be changed by changing the optimization code.


TypeError: Unexpected keyword argument passed to optimizer: df

In [104]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

df_dataset = dataset_train.convert_to_dataframe()

print(df_dataset[0].shape)

RF = RandomForestClassifier(n_estimators=1100)

PR = inprocessing.PrejudiceRemover()

PR.fit(dataset_train.features)
PR.transform(dataset_train)

'''metrics = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, RF)
df = pd.DataFrame(metrics, columns=["No Intervention"])
df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
            "false negative rate difference", "error rate difference",
            "false discovery rate difference", "false omission rate difference",
            "stat parity difference", "priv base rate", "unpriv base rate"]
df["ART"] = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, RF, ART)'''



(700, 58)


AttributeError: 'numpy.ndarray' object has no attribute 'features'

In [74]:
tf.compat.v1.disable_eager_execution()
dataset = BankDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['day_of_week'] #ignore sex-related stuff
)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

sess1 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var1", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_1:
    AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_1, sess=sess1, debias=False)

    AD.fit(dataset_train)
    dataset_test_pred = AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    AD_metrics = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

sess1.close()

metrics = AD_metrics
df = pd.DataFrame(metrics, columns=["No Intervention"])
df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
            "false negative rate difference", "error rate difference",
            "false discovery rate difference", "false omission rate difference",
            "stat parity difference", "priv base rate", "unpriv base rate"]

sess2 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var2", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_2:
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_2, sess=sess2)

    fair_AD.fit(dataset_train)
    dataset_test_pred = fair_AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    df["Adversarial Debiasing w/o dataset"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
sess2.close()

'''sess3 = tf.compat.v1.Session()
with tf.compat.v1.variable_scope("var3", reuse=tf.compat.v1.AUTO_REUSE) as scope_name_3:
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_3, sess=sess3, debias=True)

    fair_AD.fit(dataset_train.copy())
    fair_dataset_train = fair_AD.transform(dataset_train)
    fair_AD = inprocessing.AdversarialDebiasing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups,
                                           scope_name=scope_name_3, sess=sess3, debias=True)
    fair_AD.fit(fair_dataset_train)
    dataset_test_pred = fair_AD.predict(dataset_test)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    df["Adversarial Debiasing w dataset"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

sess3.close()'''

df



AttributeError: module 'tensorflow' has no attribute 'Session'

# Disregard everything underneath

In [2]:
def print_fairness_metrics(CM:ClassificationMetric):
    print(f"accuracy = {round(CM.accuracy(), 4)}")
    print(f"theil index (goal:0) = {round(CM.theil_index(), 4)}")
    print(f"binary confusion matrix = {CM.binary_confusion_matrix()}")
    print(f"consistency (goal:1) = {round(CM.consistency()[0], 4)}")
    print(f"false positive rate difference (negative:privileged bias) = {round(CM.false_positive_rate_difference(), 4)}")
    print(f"false negative rate difference (negative:privileged bias) = {round(CM.false_negative_rate_difference(), 4)}")

In [41]:
def compare_fairness_metrics_as_df(CM1:ClassificationMetric, CM2:ClassificationMetric,
                                   intervention:str) -> pd.DataFrame:
    metrics = np.array([[round(CM1.accuracy(), 4), round(CM2.accuracy(), 4)],
        [round(CM1.theil_index(), 4), round(CM2.theil_index(), 4)],
        [round(CM1.consistency()[0], 4), round(CM2.consistency()[0], 4)],
        [round(CM1.false_positive_rate_difference(), 4), round(CM2.false_positive_rate_difference(), 4)],
        [round(CM1.false_negative_rate_difference(), 4), round(CM2.false_negative_rate_difference(), 4)]]
    )
    df = pd.DataFrame(metrics, columns=["no intervention", intervention])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference"]
    return df

In [9]:
def analyze_debiasing_algos(dataset, privileged_groups, unprivileged_groups, classifier, 
                             preprocessing_algo:preprocessing = None, inprocessing_algo:inprocessing = None, 
                             postprocessing_algo:postprocessing = None):
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True)
    classifier.fit(dataset_train.features, dataset_train.labels.ravel())
    results = classifier.predict(dataset_test.features)
    
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()

    CM1 = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)

    if preprocessing_algo is not None:
        dataset_train = preprocessing_algo.fit_transform(dataset_train)
    fair_classifier = clone(classifier)
    fair_classifier.fit(dataset_train.features, dataset_train.labels.ravel())
    
    results = fair_classifier.predict(dataset_test.features)
    
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()
    
    CM2 = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    return compare_fairness_metrics_as_df(CM1, CM2, "preprocessing")
        
    '''_________________________
    RW = preprocessing.Reweighing(unprivileged_groups=unprivileged_groups,
                                 privileged_groups=privileged_groups)

    fair_dataset_train = RW.fit_transform(dataset_train)

    fair_RF = RandomForestClassifier(n_estimators=1100)
    fair_RF.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())

    results = fair_RF.predict(dataset_test.features)

    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()

    CM2 = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    compare_fairness_metrics(CM1, CM2, side_by_side=True)'''

In [36]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

In [37]:
RF = RandomForestClassifier(n_estimators=1100)
RF.fit(dataset_train.features, dataset_train.labels.ravel())

results = RF.predict(dataset_test.features)

dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

CM1 = ClassificationMetric(dataset_test,
                          dataset_test_pred,
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)

RW = preprocessing.LFR(unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

fair_dataset_train = RW.fit_transform(dataset_train)

fair_RF = RandomForestClassifier(n_estimators=1100)
fair_RF.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())

results = fair_RF.predict(dataset_test.features)

dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

CM2 = ClassificationMetric(dataset_test,
                          dataset_test_pred,
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)

In [43]:
print("German Dataset metrics")
compare_fairness_metrics_as_df(CM1, CM2, "reweighing")
print(CM1.binary_confusion_matrix(), CM2.binary_confusion_matrix())

German Dataset metrics
{'TP': 192.0, 'FP': 51.0, 'TN': 40.0, 'FN': 17.0} {'TP': 209.0, 'FP': 91.0, 'TN': 0.0, 'FN': 0.0}
